# Air Quality and Respiratory Disease
## Step1. Data Cleaning

### **Data:**

In this project, I will integrate three different types of datasets: air pollutants emissions, wildfires in Canada, and Canadian population estimates. Original data sources are as follows:
- **Air Pollution Emissions Across Provinces**: [Canada's Air Pollutant Emissions Inventory](https://data.ec.gc.ca/data/substances/monitor/canada-s-air-pollutant-emissions-inventory/APEI_Tables_Canada_Provinces_Territories/?lang)
- **Wildfires in Canada**: [Canadian National Fire Database](https://cwfis.cfs.nrcan.gc.ca/ha/nfdb)
- **Canadian Population Estimates**: [Population Estimates from Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901)<br>
<br>

[Government of Canada](https://www.canada.ca/en/environment-climate-change/services/air-pollution/pollutants/common-contaminants.html) identified sulphur oxides (sulfur oxides), nitrogen oxides, volatile organic compounds, particular matter, carbon monoxide and ammonia, and ground level ozone as the most common air contaminants in Canada. 

[Canada's Air Pollutant Inventory](https://data.ec.gc.ca/data/substances/monitor/canada-s-air-pollutant-emissions-inventory/APEI_Tables_Canada_Provinces_Territories/?lang) has a total of thirteen air pollutants listed. From here, I will focus on sulfur oxides, nitrogen oxides, volatile organic compounds, carbon monoxide, and ammonia; the pollutants that are marked as the most common air contaminants in Canada. 

In [2]:
# Import libraries
import pandas as pd
from functools import partial
from tqdm.notebook import tnrange 
from time import sleep
from IPython.display import clear_output

#### Air Pollution Emission Across Provinces:
Due to a large data file size, it may take a while for all data to load. A progress bar will show its progress in real time. 

In [56]:
# import data :: Air Pollutants 

# create new dataframe
d = {"Province": [],
    "Pollutant": []}
    
provinces = ["AB", "BC", "ON", "MB", "NB", "NS", "YT", "QC", "SK", "NL", "PE"]
common_pollutants = ["NH3","CO","SOX","NOX","VOC"]
provnum = 0

def import_data(province, pollutants):
    df = pd.DataFrame.from_dict(d)
    for i in pollutants:
        p = pd.read_csv("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/province-pollutants/" + province + "_" + i + ".csv")
        p_total = p.loc[p["SECTORS"] == "GRAND TOTAL"]
        p_total = p_total.drop(["Unnamed: 1", "Unnamed: 2"], axis=1)
        p_total = p_total.rename(columns={"SECTORS":"Province"})
        p_total.at[1,"Province"] = province
        p_total.insert(1,"Pollutant", i, True)
        df = pd.concat([df, p_total], ignore_index=True)
        
        for i in tnrange(100, desc='Downloading ' + i + ' data for ' + province):
            startRow = 1 + i*100
            sleep(0.01)
    
    global provnum
    if provnum > 11:
        provnum = 0
    else:
        provnum += 1
            
    print("All provincial data for " + province + " has been downloaded." + 
         " Remaining provinces:" + str(len(provinces) - provnum) + "/" + str(len(provinces)) + ".") 
        
    return df

df = pd.concat(map(partial(import_data, pollutants=common_pollutants), provinces),ignore_index=True)
clear_output(wait=True)
df.head()

,Province,Pollutant,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AB,NH3,95261.6403072018,96392.6120160835,100809.748597631,102888.757346287,108909.29619548,117873.246725767,123629.070648118,126274.333185743,...,132641.48219304642,137630.5644576695,139707.6493382899,141095.12881051924,139937.588499843,132619.48058146506,124020.85788901642,130492.08344071463,131440.5311861856,133161.78812024335
1,AB,CO,1810389.43111,1674240.47593966,1694347.04215079,1670384.18553975,1713860.13181427,1699241.95442997,1707362.43286568,1747506.53522988,...,1015160.0103461307,1020173.0616697504,1044806.4504109875,1083451.1163415606,985516.9393906805,950437.9647403281,1009269.6788974762,1001193.5531940853,992062.7619323295,890235.0708604576
2,AB,SOX,512405.17695316,524144.18481018,564717.776802336,571473.902885045,594900.160427548,569212.579973766,556679.129617835,522989.264001654,...,343134.3155222252,333877.81221442844,314187.69199869514,291592.47385681805,259725.37296679677,239568.79130380912,241439.9996413067,225758.93672193997,221287.8184768282,182536.32978225523
3,AB,NOX,613284.286308138,585705.325771787,609463.591567837,640884.40812074,692547.095770932,718124.92776215,749396.467537041,815873.317241737,...,691938.4815302572,649050.7227761747,643710.2914325446,656735.6508195029,630979.6246838769,603268.9203695664,627271.8514718515,625976.446078691,625636.7223761571,567697.196882858
4,AB,VOC,643752.225882095,624633.776990753,642477.562274331,654384.418252455,669557.452773095,679792.446419822,708522.408799455,678362.261827271,...,478680.32202698546,518824.5195309372,556411.0771793935,572615.3277760863,517421.8223597623,476740.12370002636,474301.4225637558,501074.8373153089,492636.6127706006,456932.64826396375


#### Canadian Population Estimates:
The [Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901) dataset includes information on monthly population estimates from January 1990 to January 2022. I will focus our analysis on the data gathered between 1990 and 2020. To do so, I drop all the data after December 2020. 

I want to look at provincial emissions of air pollutants *per capita*. To achieve this, first I need to combine the air pollutants and population per capita dataframes that have been already downloaded and imported. To combine the two dataframes, I rename the `GEO` variable in the population per capita dataframe `Province`. 

Once both data are cleaned, I merge the two datasets based on `Year` and `Province`. The resulting dataset will contain provincial air pollutant emissions and population estimates for every year between 1990 and 2020. Then I divide air pollutant emissions `Value` by population `Per Capita` to get emission `By Capita`. 

In [50]:
# data cleaning for air pollutant data
per_capita = pd.read_csv("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/per-capita.csv")
air_pollutant = df.melt(id_vars = ["Province", "Pollutant"], var_name = "Year", value_name = "Value")
air_pollutant["Year"] = air_pollutant["Year"].astype(int)

# data cleaning for per capita data
prov = {"Canada": "CAN", "Newfoundland and Labrador": "NL", "Prince Edward Island":"PE",
       "Nova Scotia":"NS", "New Brunswick":"NB", "Quebec":"QC", "Ontario":"ON",
       "Manitoba":"MB", "Alberta":"AB", "Saskatchewan":"SK", "British Columbia":"BC",
       "Yukon":"YK", "Northwest Territories":"NT", "Nunavut":"NU"}
per_capita = per_capita.replace(prov)
per_capita["Year"]=pd.to_datetime(per_capita["REF_DATE"]).dt.year
per_capita=per_capita.groupby(["Year","GEO"], as_index=False).mean()
per_capita = per_capita[["Year", "GEO", "VALUE"]]
per_capita = per_capita.loc[per_capita["Year"] < 2021]
per_capita = per_capita.rename(columns= {"GEO":"Province", "VALUE":"Per Capita"})

# merge the air pollutant and per capita datsets
pollutant_capita = pd.merge(air_pollutant, per_capita, on=["Year", "Province"])

# divide 
pollutant_capita["Value"] = pd.to_numeric(pollutant_capita["Value"])
pollutant_capita["By Capita"] = pollutant_capita["Value"].div(pollutant_capita["Per Capita"])
pollutant_capita.head()

,Province,Pollutant,Year,Value,Per Capita,By Capita
0,AB,NH3,1990,9.526164e+04,2540901.75,0.037491
1,AB,CO,1990,1.810389e+06,2540901.75,0.712499
2,AB,SOX,1990,5.124052e+05,2540901.75,0.201663
3,AB,NOX,1990,6.132843e+05,2540901.75,0.241365
4,AB,VOC,1990,6.437522e+05,2540901.75,0.253356


In [49]:
# calculate mean per capita emission by pollutants. 
pollutant_capita_mean = pollutant_capita[["Province", "Pollutant", "By Capita"]].groupby(["Province", "Pollutant"]).mean()
pollutant_capita_mean = pollutant_capita_mean.rename(columns={"By Capita": "Mean"}).reset_index()
pollutant_capita_mean

# calculate total emission of each pollutants from all provinces.  
pollutant_capita_sum = pollutant_capita[["Province", "Pollutant", "Value"]].groupby(["Province", "Pollutant"]).sum()
pollutant_capita_sum = pollutant_capita_sum.rename(columns={"Value": "Total Emission"}).reset_index()
pollutant_capita_sum

pollutant_capita_mean.head()

,Province,Pollutant,Mean,Per Capita
0,AB,CO,0.426834,3.398268e+06
1,AB,NH3,0.038578,3.398268e+06
2,AB,NOX,0.209131,3.398268e+06
3,AB,SOX,0.131820,3.398268e+06
4,AB,VOC,0.179616,3.398268e+06


#### Wildfires in Canada:
The [Canadian National Fire Database](https://cwfis.cfs.nrcan.gc.ca/ha/nfdb) has various information on wildfires in Canada. I will look into the total number of wildfires and the total area burned by province. Using this information, I can observe whether `Number of Fire` or `Area Burned (hectre)` correlates better with PM2.5 emissions. 

In [53]:
def organize(df, title):
    df = pd.read_csv(df)
    df = df[df["Jurisdiction"].notna()]
    df = df.melt(id_vars = ["Jurisdiction", "Month"],
                 var_name = "Year",value_name = "Number")
    df = df.fillna(0)
    df = df.groupby(["Jurisdiction"])["Number"].sum()
    df = pd.Series(df).to_frame().reset_index()
    df = df.replace(prov)
    df = df.rename(columns={"Number":str(title),
                                     "Jurisdiction":"Province"})
    return df

firenum = organize("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/wildfire/total_fire.csv", "Number of Fire")
firearea = organize("https://raw.githubusercontent.com/jlee2843/portfolio/main/Air-quality-prediction/data/wildfire/total_area_burnt.csv", "Area Burned (hectre)")
fire = pd.merge(firenum, firearea, on=["Province"])
fire

,Province,Number of Fire,Area Burned (hectre)
0,AB,38015.0,5.923754e+06
1,BC,56698.0,5.123188e+06
2,MB,10953.0,6.672828e+06
3,NB,9985.0,2.908940e+04
4,NL,2937.0,4.148917e+05
5,NT,7295.0,1.859678e+07
6,NS,8841.0,2.159480e+04
7,ON,35041.0,4.865412e+06
8,Parks Canada,2512.0,2.401450e+06
9,PE,11.0,2.160128e+01


On below, I used the same method that I previsouly developed to gather provincial data for air pollutant emissions. To only focus our attention on PM2.5, I trimmed the data to look at total PM2.5 emissions across provinces.

In [54]:
common_pollutants= ["PM25"]
provnum = 0
pm25 = pd.concat(map(partial(import_data, pollutants=common_pollutants), provinces), ignore_index=True)
pm25 = pm25.melt(id_vars = ["Province", "Pollutant"], var_name = "Year", value_name = "Total Emission")
pm25["Total Emission"] = pd.to_numeric(pm25["Total Emission"])
pm25 = pm25.groupby(["Province"])["Total Emission"].sum()
pm25 = pd.Series(pm25).to_frame().reset_index()
pm25_fire = pd.merge(pm25, fire, on=["Province"])
clear_output(wait=True)
pm25_fire

,Province,Total Emission,Number of Fire,Area Burned (hectre)
0,AB,1.330822e+07,38015.0,5.923754e+06
1,BC,2.600960e+06,56698.0,5.123188e+06
2,MB,3.077027e+06,10953.0,6.672828e+06
3,NB,6.211207e+05,9985.0,2.908940e+04
4,NL,4.874982e+05,2937.0,4.148917e+05
5,NS,6.455169e+05,8841.0,2.159480e+04
6,ON,5.625880e+06,35041.0,4.865412e+06
7,PE,1.235335e+05,11.0,2.160128e+01
8,QC,3.890464e+06,21222.0,8.173951e+06
9,SK,1.422373e+07,16964.0,1.488577e+07


I have completed the first step in data analysis, which would be data cleaning. Now I move onto visualize our findings so it would be easier for the readers to interpret. 